In [62]:
# imports
import gc
import os
import sys
from io import BytesIO
import time
from IPython.display import Image, display as image, display
from IPython.display import HTML
import requests
from PIL import Image
import numpy as np
import statistics as st
import datetime
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import rc
import pandas as pd
import dask.dataframe as dd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows',120)
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.gridspec as gridspec
import seaborn as sns
from termcolor import colored
%matplotlib inline
sns.set_style('darkgrid')
import plotly.express as px
import plotly.figure_factory as ff
hcmap = sns.blend_palette(['yellowgreen','orange','orangered','red'],n_colors=12,as_cmap=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.feature_selection import mutual_info_regression
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split as split
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV as Grid
import xgboost as xgb
from sklearn.metrics import (f1_score, accuracy_score, balanced_accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix)

In [63]:
input_data_path = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        input_data_path.append(os.path.join(dirname, filename))
input_data_path

['/kaggle/input/nfl-big-data-bowl-2023/players.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/week6.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/week2.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/pffScoutingData.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/week3.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/week8.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/games.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/week5.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/week7.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/week1.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/week4.csv',
 '/kaggle/input/nfl-big-data-bowl-2023/plays.csv']

In [64]:
games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2023/games.csv')
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2023/plays.csv')
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2023/players.csv')
PFFScoutingData = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2023/pffScoutingData.csv')

In [65]:
games.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2021090900,2021,1,09/09/2021,20:20:00,TB,DAL
1,2021091200,2021,1,09/12/2021,13:00:00,ATL,PHI
2,2021091201,2021,1,09/12/2021,13:00:00,BUF,PIT
3,2021091202,2021,1,09/12/2021,13:00:00,CAR,NYJ
4,2021091203,2021,1,09/12/2021,13:00:00,CIN,MIN


In [66]:
PFFScoutingData.head()

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021090900,97,39985,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0


In [67]:
wk1 = pd.read_csv( '/kaggle/input/nfl-big-data-bowl-2023/week1.csv')
wk2 = pd.read_csv( '/kaggle/input/nfl-big-data-bowl-2023/week2.csv')
wk3 = pd.read_csv( '/kaggle/input/nfl-big-data-bowl-2023/week3.csv')
wk4 = pd.read_csv( '/kaggle/input/nfl-big-data-bowl-2023/week4.csv')
wk5 = pd.read_csv( '/kaggle/input/nfl-big-data-bowl-2023/week5.csv')
wk6 = pd.read_csv( '/kaggle/input/nfl-big-data-bowl-2023/week6.csv')
wk7 = pd.read_csv( '/kaggle/input/nfl-big-data-bowl-2023/week7.csv')
wk8 = pd.read_csv( '/kaggle/input/nfl-big-data-bowl-2023/week8.csv')


In [68]:
DFs = [wk1, wk2, wk3, wk4, wk5, wk6, wk7, wk8]
tracking = pd.concat( DFs,axis=0,join="outer",ignore_index=False,copy=True,)
tracking.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8314178 entries, 0 to 978948
Data columns (total 16 columns):
 #   Column         Dtype  
---  ------         -----  
 0   gameId         int64  
 1   playId         int64  
 2   nflId          float64
 3   frameId        int64  
 4   time           object 
 5   jerseyNumber   float64
 6   team           object 
 7   playDirection  object 
 8   x              float64
 9   y              float64
 10  s              float64
 11  a              float64
 12  dis            float64
 13  o              float64
 14  dir            float64
 15  event          object 
dtypes: float64(9), int64(3), object(4)
memory usage: 1.1+ GB


In [69]:
tracking.head(5)

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None


In [70]:
tracking[tracking.nflId.isnull()].head(8)

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
946,2021090900,97,NaN,1,2021-09-10T00:26:31.100,NaN,football,right,42.92,24.04,0.00,0.00,0.00,NaN,NaN,None
947,2021090900,97,NaN,2,2021-09-10T00:26:31.200,NaN,football,right,42.23,23.91,0.25,0.12,0.70,NaN,NaN,None
948,2021090900,97,NaN,3,2021-09-10T00:26:31.300,NaN,football,right,42.17,23.89,0.19,0.73,0.07,NaN,NaN,None
949,2021090900,97,NaN,4,2021-09-10T00:26:31.400,NaN,football,right,42.10,23.89,0.04,1.86,0.07,NaN,NaN,None
950,2021090900,97,NaN,5,2021-09-10T00:26:31.500,NaN,football,right,41.82,23.91,1.19,6.28,0.28,NaN,NaN,None
951,2021090900,97,NaN,6,2021-09-10T00:26:31.600,NaN,football,right,41.56,23.92,2.52,10.03,0.26,NaN,NaN,ball_snap
952,2021090900,97,NaN,7,2021-09-10T00:26:31.700,NaN,football,right,41.07,23.94,3.20,7.80,0.49,NaN,NaN,None
953,2021090900,97,NaN,8,2021-09-10T00:26:31.800,NaN,football,right,40.71,23.95,3.93,6.74,0.36,NaN,NaN,None


In [71]:
del DFs
gc.collect()

717

In [74]:
ball_df = tracking[tracking.team == 'football'].copy()
tracking = tracking[tracking.team != 'football'].copy()
ball_df.drop(columns=['o', 'dir', 'nflId', 'jerseyNumber'],inplace=True)
ball_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361486 entries, 946 to 978948
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   gameId         361486 non-null  int64  
 1   playId         361486 non-null  int64  
 2   frameId        361486 non-null  int64  
 3   time           361486 non-null  object 
 4   team           361486 non-null  object 
 5   playDirection  361486 non-null  object 
 6   x              361486 non-null  float64
 7   y              361486 non-null  float64
 8   s              361486 non-null  float64
 9   a              361486 non-null  float64
 10  dis            361486 non-null  float64
 11  event          361486 non-null  object 
dtypes: float64(5), int64(3), object(4)
memory usage: 35.9+ MB


In [77]:
 tracking.isnull().sum()

gameId           0
playId           0
nflId            0
frameId          0
time             0
jerseyNumber     0
team             0
playDirection    0
x                0
y                0
s                0
a                0
dis              0
o                0
dir              0
event            0
dtype: int64

In [83]:
tracking[tracking.frameId.duplicated()]

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
43,2021090900,97,35481.0,1,2021-09-10T00:26:31.100,87.0,TB,right,41.08,29.15,0.31,0.43,0.03,35.96,238.77,None
44,2021090900,97,35481.0,2,2021-09-10T00:26:31.200,87.0,TB,right,41.34,29.45,0.38,1.02,0.40,102.09,29.56,None
45,2021090900,97,35481.0,3,2021-09-10T00:26:31.300,87.0,TB,right,41.53,29.39,0.61,1.10,0.20,68.16,71.57,None
46,2021090900,97,35481.0,4,2021-09-10T00:26:31.400,87.0,TB,right,41.53,29.38,0.53,0.76,0.01,73.06,82.44,None
47,2021090900,97,35481.0,5,2021-09-10T00:26:31.500,87.0,TB,right,41.54,29.36,0.97,1.52,0.03,76.60,75.14,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978886,2021110100,4433,53449.0,54,2021-11-02T03:20:26.500,89.0,NYG,right,49.44,48.85,1.43,3.20,0.14,270.45,2.03,None
978887,2021110100,4433,53449.0,55,2021-11-02T03:20:26.600,89.0,NYG,right,49.43,48.99,1.34,2.85,0.13,273.20,349.13,None
978888,2021110100,4433,53449.0,56,2021-11-02T03:20:26.700,89.0,NYG,right,49.40,49.11,1.32,2.42,0.13,275.46,338.05,None
978889,2021110100,4433,53449.0,57,2021-11-02T03:20:26.800,89.0,NYG,right,49.33,49.22,1.29,2.18,0.13,277.09,326.21,None
